In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
# import pathlib
# dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
# data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = '/home/hrishi/FMS/ds/train_ds/sort_machine'

In [3]:
batch_size = 8
img_height = 52
img_width = 52

In [4]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1383 files belonging to 3 classes.
Using 1107 files for training.


In [5]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1383 files belonging to 3 classes.
Using 276 files for validation.


In [6]:
class_names = val_ds.class_names
print(class_names)

['empty', 'orange', 'white']


In [7]:
def rescale(inp,gt):
    return (inp/255.0,gt)

In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.map(rescale).shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(rescale).prefetch(buffer_size=AUTOTUNE)

In [9]:
num_classes = 3

model = Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu',input_shape=(img_height, img_width, 3)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 52, 52, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0

In [12]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/5
139/139 [==============================] - 18s 34ms/step - loss: 0.1089 - accuracy: 0.9503 - val_loss: 3.7830e-05 - val_accuracy: 1.0000
Epoch 2/5
139/139 [==============================] - 3s 18ms/step - loss: 3.5799e-04 - accuracy: 1.0000 - val_loss: 1.8165e-05 - val_accuracy: 1.0000
Epoch 3/5
139/139 [==============================] - 3s 19ms/step - loss: 1.9264e-05 - accuracy: 1.0000 - val_loss: 1.3791e-05 - val_accuracy: 1.0000
Epoch 4/5
139/139 [==============================] - 3s 18ms/step - loss: 1.2612e-05 - accuracy: 1.0000 - val_loss: 1.0254e-05 - val_accuracy: 1.0000
Epoch 5/5
139/139 [==============================] - 3s 18ms/step - loss: 9.6106e-06 - accuracy: 1.0000 - val_loss: 6.6831e-06 - val_accuracy: 1.0000


In [13]:
model.save('/home/hrishi/FMS/classification/3_class_model')

INFO:tensorflow:Assets written to: /home/hrishi/FMS/classification/3_class_model/assets


In [14]:
trained_model  = tf.keras.models.load_model('/home/hrishi/FMS/classification/3_class_model')

In [15]:
params = tf.experimental.tensorrt.ConversionParams(precision_mode='FP16',maximum_cached_engines=100)

In [16]:
converter = tf.experimental.tensorrt.Converter(input_saved_model_dir='/home/hrishi/FMS/classification/3_class_model', conversion_params=params)

INFO:tensorflow:Linked TensorRT version: (8, 0, 1)
INFO:tensorflow:Loaded TensorRT version: (8, 0, 1)


In [17]:
converter.convert()

<ConcreteFunction pruned(conv2d_input) at 0x7EE42F2CC0>

In [18]:
def my_input_fn():
  for _ in range(100):
    inp1 = tf.random.normal((1,52,52,3),dtype=tf.float32)
    # inp1 = np.random.normal((52,52,3))
    yield [inp1]

In [19]:
converter.build(input_fn=my_input_fn)

In [20]:
converter.save('/home/hrishi/FMS/classification/3_class_model_fp16')

INFO:tensorflow:Assets written to: /home/hrishi/FMS/classification/3_class_model_fp16/assets


In [15]:


# sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
# sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)
orange_ball_path = '/home/hrishi/FMS/ds/train_ds/orange/1.jpg'
img = tf.keras.preprocessing.image.load_img(
    orange_ball_path, target_size=(img_height, img_width)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = img_array/255.0
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to orange with a 100.00 percent confidence.


# Inference

In [ ]:
import cv2